In [1]:
import torch
import matplotlib as plt
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
import os

folder_path = "C:/Users/zacca/Python/DL_Proj_Models/dataset/pathtools_gleason_grading/masks"
files = os.listdir(folder_path)

In [3]:
class_1 = []
class_2 = []
class_3 = []
class_4 = []
class_5 = []
class_none = []

for file_name in tqdm(files):
    file_path = os.path.join(folder_path, file_name)
    image = Image.open(file_path)
    image_data = image.getdata()

    if 1 in image_data:
        class_1.append(file_name)

    if 2 in image_data:
        class_2.append(file_name)

    if 3 in image_data:
        class_3.append(file_name)

    if 4 in image_data:
        class_4.append(file_name)

    if 5 in image_data:
        class_5.append(file_name)

    if all(x == 0 for x in image_data):
        class_none.append(file_name)

100%|████████████████████████████████████████████████████████████████████████████| 51468/51468 [09:45<00:00, 87.90it/s]


In [4]:
print(len(class_1))
print(len(class_2))
print(len(class_3))
print(len(class_4))
print(len(class_5))
print(len(class_none))

66
3615
4305
11183
0
33205


In [39]:
import random

num_augmentations = 30

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=90)
])

seeds = []

for i in range(num_augmentations):
    seeds.append(random.randint(0, 1000))

seeds

[266,
 357,
 802,
 324,
 148,
 86,
 989,
 685,
 410,
 451,
 487,
 11,
 452,
 261,
 578,
 580,
 57,
 609,
 478,
 540,
 349,
 697,
 483,
 177,
 621,
 530,
 835,
 132,
 79,
 477]

In [40]:
mask_folder_path = "C:/Users/zacca/Python/DL_Proj_Models/dataset/pathtools_gleason_grading/masks"
target_folder_path = "C:/Users/zacca/Python/DL_Proj_Models/dataset/pathtools_gleason_grading/images"

for file_name in class_1:
    mask_file_path = os.path.join(mask_folder_path, file_name)
    target_file_path = os.path.join(target_folder_path, file_name)
    
    mask_image = Image.open(mask_file_path)
    target_image = Image.open(target_file_path)

    for i in range(num_augmentations):
        torch.manual_seed(seeds[i])
        augmented_mask_image = transform(mask_image)
        torch.manual_seed(seeds[i])
        augmented_target_image = transform(target_image)
        
        # Save the augmented images
        augmented_mask_file_path = os.path.join(mask_folder_path, f"augmented_{file_name[:-4]}_{i}.jpg")
        augmented_target_file_path = os.path.join(target_folder_path, f"augmented_{file_name[:-4]}_{i}.jpg")
        
        augmented_mask_image.save(augmented_mask_file_path)
        augmented_target_image.save(augmented_target_file_path)

In [41]:

folder_path = "C:/Users/zacca/Python/DL_Proj_Models/dataset/pathtools_gleason_grading/masks"
files = os.listdir(folder_path)



class_11 = []
class_21 = []
class_31 = []
class_41 = []

class_none1 = []
extra = []
llist = [0,1,2,3,4,5]
for file_name in tqdm(files):
    file_path = os.path.join(folder_path, file_name)
    image = Image.open(file_path)
    image_data = image.getdata()

    if set(image_data) & set([5,6,7,8]):
        continue
        
    if 1 in image_data:
        class_11.append(file_name)

    if 2 in image_data:
        class_21.append(file_name)

    if 3 in image_data:
        class_31.append(file_name)

    if 4 in image_data:
        class_41.append(file_name)
        
    if all(x == 0 for x in image_data):
        class_none1.append(file_name)

    extra.append(any(num not in llist for num in image_data))
    unique_numbers = list(set(extra))


print(len(class_none1))
print(len(class_11))
print(len(class_21))
print(len(class_31))
print(len(class_41))

100%|████████████████████████████████████████████████████████████████████████████| 53448/53448 [13:47<00:00, 64.59it/s]

33243
1063
4544
4863
11496


In [45]:
augmented_training_list = class_11 + class_21 + class_31 + random.sample(class_41, 5000) + random.sample(class_none1, 2000)

In [46]:
from pathlib import Path

masks = [str(Path(mask_folder_path) / file) for file in augmented_training_list]  # Combine paths
imgs = [str(Path(target_folder_path) / file) for file in augmented_training_list]  # Combine paths
torch.save((imgs, masks), 'train_data_augmented.pth')

In [17]:
def get_weights(dis):
    total_samples = sum(dis)

    # Calculate inverse class frequencies
    inverse_frequencies = [total_samples / (class_samples * len(dis)) for class_samples in dis]

    # Normalize inverse frequencies to sum up to 1
    class_weights = [freq / sum(inverse_frequencies) for freq in inverse_frequencies]

    return class_weights

In [47]:
dis = [2000, 1063, 4544, 4863, 5000]
get_weights(dis)

[0.24196218826831767,
 0.45524400426776607,
 0.106497442019506,
 0.09951149013708314,
 0.09678487530732706]

In [21]:

folder_path = "C:/Users/zacca/Python/DL_Proj_Models/dataset/pathtools_gleason_grading/masks"
files = os.listdir(folder_path)
uniques = []
for file_name in tqdm(files):
    file_path = os.path.join(folder_path, file_name)
    image = Image.open(file_path)
    image_data = image.getdata()

    unique_numbers = list(set(image_data))
    uniques += [num for num in unique_numbers if num not in uniques]

uniques

100%|███████████████████████████████████████████████████████████████████████████| 52458/52458 [01:27<00:00, 596.55it/s]


[0, 2, 1, 3, 4, 5, 6, 7, 8]

In [35]:
t = [1,2,3,4]
h = [4,5,6,7]
set(t) & set(h)

{4}